# MVE Axiom Prediction

In [1]:
%env CUDA_VISIBLE_DEVICES=""

env: CUDA_VISIBLE_DEVICES=""


# TODO




* Try to get a reasonable score?
* TODO add batchnorm
* TODO add residual connections
* TODO add other directions?

# Imports

In [2]:
import torch
from torch.nn import Embedding
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, Linear
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import to_undirected, ToUndirected

/home/eholden/.pyenv/versions/3.7.5/lib/python3.7/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import sys, os
from pathlib import Path
sys.path.append(str(Path(os.path.abspath('')).parent))

import config
from dataset import get_data_loader, BenchmarkType

## CONSTANTS

In [4]:
#TRAIN_ID = '../id_files/train.txt'
TRAIN_ID = '../id_files/validation.txt'

VAL_ID = '../id_files/validation.txt'
BENCHMARK_TYPE = BenchmarkType('deepmath')

In [5]:
torch.manual_seed(1234567)

## Load dataset

In [6]:
ToUndirected

torch_geometric.transforms.to_undirected.ToUndirected

In [7]:
train_data = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, transform=ToUndirected())
next(iter(train_data))

Dataset: TorchMemoryDataset(2465)


DataBatch(x=[36912], edge_index=[2, 120812], premise_index=[842], conjecture_index=[64], name=[64], y=[842], batch=[36912], ptr=[65])

In [8]:
#transform = ToUndirected()
transform = None

In [9]:
train_data = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, transform=transform)
val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, transform=transform)

Dataset: TorchMemoryDataset(2465)
Dataset: TorchMemoryDataset(2465)


### Small check on a datapoint

In [10]:
next(iter(train_data))

DataBatch(x=[40357], edge_index=[2, 66926], premise_index=[842], conjecture_index=[64], name=[64], y=[842], batch=[40357], ptr=[65])

In [11]:
data = next(iter(train_data))[0]
print(data)
print(data.keys)
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
print(data.has_isolated_nodes())
print(data.is_directed())


Data(x=[771], edge_index=[2, 1423], premise_index=[6], conjecture_index=[1], name='t41_setwop_2', y=[6])
['edge_index', 'name', 'x', 'premise_index', 'conjecture_index', 'y']
771
1423
1
True
True


## Model

In [12]:
#exp_params = {flow: "target_to_source"}

In [13]:
def get_dense_network(hidden_dim, task='binary', dropout_rate=0.0):
    if task == 'binary':
        return nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Dropout(dropout_rate), 
            nn.Linear(hidden_dim, 1)) # Two layer dense output network
    

In [14]:
class GNNStack(torch.nn.Module):
    def __init__(self, hidden_dim, num_convolutional_layers, dropout_rate=0.0, task='node'):
        
        super(GNNStack, self).__init__()
        #self.flow = 'source_to_target'
        self.flow = 'target_to_source'
        
        self.task = task
        self.num_convolutional_layers = num_convolutional_layers
        self.dropout_rate = dropout_rate
        
        # Add embedding layer
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_dim)

        # Add convolutional layers
        self.convs = nn.ModuleList()
        for _ in range(self.num_convolutional_layers): 
            self.convs.append(self.build_conv_model(hidden_dim, hidden_dim))
        
        # Add normalisation layers (not after last convolution)
        self.lns = nn.ModuleList()
        for _ in range(self.num_convolutional_layers -1): 
            self.lns.append(nn.LayerNorm(hidden_dim))
            
        # post-message-passing
        self.post_mp = get_dense_network(hidden_dim, dropout_rate=self.dropout_rate)
        
        
        #if not (self.task == 'node' or self.task == 'graph'):
        #    raise RuntimeError('Unknown task.')

        
    def build_conv_model(self, input_dim, hidden_dim):
        # refer to pytorch geometric nn module for different implementation of GNNs.
        #if self.task == 'node':
        return pyg_nn.GCNConv(input_dim, hidden_dim, flow=self.flow)
        #else:
        #    return pyg_nn.GINConv(nn.Sequential(nn.Linear(input_dim, hidden_dim),
        #                          nn.ReLU(), nn.Linear(hidden_dim, hidden_dim)))

    def forward(self, data):
        x, edge_index, premise_index = data.x, data.edge_index, data.premise_index    

        
        x = self.node_embedding(x)
        
        #if data.num_node_features == 0:
        #  x = torch.ones(data.num_nodes, 1)

        for i in range(self.num_convolutional_layers):
            x = self.convs[i](x, edge_index)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
            if not i == self.num_convolutional_layers - 1: # Apply normalisation
                x = self.lns[i](x)

        #if self.task == 'graph':
        #    x = pyg_nn.global_mean_pool(x, batch)

        # TODO should this be combined?
        x = x[premise_index]
        x = self.post_mp(x)
        x = x.squeeze(-1)
        return x

        #
        # return emb, F.log_softmax(x, dim=1) FIXME

    def loss(self, pred, label):
        return F.nll_loss(pred, label)
    
    
model = GNNStack(hidden_dim=32, num_convolutional_layers=3, dropout_rate=0.25, task='graph')

In [15]:
'''
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_channels)


        self.conv1 = GCNConv(hidden_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        
        self.linear = Linear(hidden_channels, 1)
        

    def forward(self, input_batch):
        
        x = input_batch.x
        edge_index = input_batch.edge_index
        premise_index = input_batch.premise_index    

        x = self.node_embedding(x)

        x = self.conv1(x, edge_index)
        x = x.relu()
        
        x = F.dropout(x, p=0.5, training=self.training) # TODO add dropout parameter?
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        
        x = x[premise_index]
        x = self.linear(x)
        
        # Remove inner axis
        x = x.squeeze(-1)
        
    
        return x

    
model = GCN(hidden_channels=16)
print(model)
'''

'\nclass GCN(torch.nn.Module):\n    def __init__(self, hidden_channels):\n        super().__init__()\n        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_channels)\n\n\n        self.conv1 = GCNConv(hidden_channels, hidden_channels)\n        self.conv2 = GCNConv(hidden_channels, hidden_channels)\n        \n        self.linear = Linear(hidden_channels, 1)\n        \n\n    def forward(self, input_batch):\n        \n        x = input_batch.x\n        edge_index = input_batch.edge_index\n        premise_index = input_batch.premise_index    \n\n        x = self.node_embedding(x)\n\n        x = self.conv1(x, edge_index)\n        x = x.relu()\n        \n        x = F.dropout(x, p=0.5, training=self.training) # TODO add dropout parameter?\n        \n        x = self.conv2(x, edge_index)\n        x = x.relu()\n        \n        x = x[premise_index]\n        x = self.linear(x)\n        \n        # Remove inner axis\n        x = x.squeeze(-1)\n        \n    \n        return x\n\n

## Train and Test

In [16]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
#criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

In [17]:
def train():
    
    model.train()
    
    for data in train_data:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass. TODO change this

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

        
        
def test(loader):
    
    model.eval()

    correct = 0
    total_samples = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)
        pred = torch.sigmoid(out).round().long()
        
        correct += data.y.eq(pred).sum().item()
        
        total_samples += len(pred)

    return correct / total_samples  # Derive ratio of correct predictions.



In [18]:

for epoch in range(1, 3):
    train()
    train_acc = test(train_data)
    test_acc = test(val_data)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6499, Test Acc: 0.6499
Epoch: 002, Train Acc: 0.6488, Test Acc: 0.6488
